In [ ]:
# Purpose: speed up proof of auth data pulls
# NB: Last version committed hasn't been tested (no VPN right now), but it was broken already

In [ ]:
# Get going with libs and database connections
import pandas as pd
import sys
import datetime
# import AHSql

In [ ]:
from os.path import expanduser
home = expanduser("~")
sys.path.insert(0, home+'/Activehours/data-analysis/Utilities')

In [ ]:
sys.path.append('C:\\Users\\imtys\\Documents\\GitHub\\data-analysis\\Utilities')

In [ ]:
import AHSql

In [ ]:
ahsql = AHSql.AHSql()

In [ ]:
clientsql = AHSql.AHSql("clientstate")

In [ ]:
# Set up data pull
firstname = 'Wilson'
lastname = 'Instervil'
datestring = '2018-04-24' # should be day after restore date listed by CRB (file date -> posted)

In [ ]:
transactionaldata = ahsql.sqlDataFromText("""
SELECT 
    a.userid, u.UserName, identifiers.AccountNumber, t.Amount CashoutAmount,
    a.tip, a.CreatedOn, a.RestoreDate, ld.UDID, ld.TmStamp, ld.OS, d.DeviceModel
FROM (
    SELECT t.TransactionId, mmft.AccountNumber, max(ld.LoginID) LoginId
    FROM Activations a (nolock)
    join transactions t (nolock)
    on t.ActivationID = a.ActivationID
    inner join MoneyMovement.TransactionsToFundsTransfers mmttft (nolock)
    on mmttft.TransactionId = t.TransactionId
    inner join MoneyMovement.FundsTransfers mmft (nolock)
    on mmft.FundsTransferId = mmttft.FundsTransferId
    join users u (nolock)
    on u.userid = a.UserID
    inner join LoginDetails ld (nolock)
    on ld.UserId = a.UserId and ld.TmStamp <= a.CreatedOn
    where --AccountNumber = '021214860000865'
        FirstName = '{}' and LastName = '{}'
    and mmft.PostingDate in ('{}') -- Date after what the bank lists (file v posted)
    and TransactionTypeID in (3,4,7)
    and t.StatusID in (2,3)
    and t.Amount != Tip
    AND ld.IsSuccessfulLogin = 1
    group by t.TransactionId, mmft.AccountNumber
) identifiers
inner join Transactions t (nolock)
on identifiers.TransactionId = t.TransactionId
inner join Activations a (nolock)
on t.ActivationId = a.ActivationId
inner join LoginDetails ld (nolock)
on identifiers.LoginId = ld.LoginID
inner join users u (nolock)
on u.userid = a.userid
inner join devices d (nolock)
on d.udid = ld.UDID
order by 3""".format(firstname, lastname, datestring))
transactionaldata.head()

In [ ]:
# This query is usually accurate, but not always. It incorrectly assumes that the locationid column is ordered by time, 
# sequentially. That's usually true, but not always.
data_to_add = []
for index, row in transactionaldata.iterrows():
    client_data = clientsql.sqlDataFromText("""
    SELECT l.LocationId, n.carriername, n.IpAddress, n.NetworkId, n.NetworkType, n.Status, n.TimeStamp
     FROM clientstate.Locations l
     INNER JOIN (
         SELECT MAX(l.LocationId) LocationId
         FROM clientstate.Locations l
         LEFT JOIN clientstate.ClientState cs
           ON l.LocationId = cs.LocationId
         LEFT JOIN clientstate.Networks n
           ON cs.NetworkId = n.ClientStateNetworkId
         WHERE l.UserId = {userid}
         AND DATE_ADD(l.TimeStamp, INTERVAL -5 HOUR) < '{date}'
     ) mostrecent
     ON mostrecent.LocationId = l.LocationId
     LEFT JOIN clientstate.ClientState cs
       ON l.LocationId = cs.LocationId
     LEFT JOIN clientstate.Networks n
       ON cs.NetworkId = n.ClientStateNetworkId
     WHERE l.UserId = {userid}
     AND DATE_ADD(l.TimeStamp, INTERVAL -5 HOUR) < '{date}'
     LIMIT 20""".format(userid = row['userid'], date = row['TmStamp'])
    )
    #print(client_data)
    carrier = 'WiFi' if client_data['NetworkType'][0].lower() == 'wifi' else client_data['NetworkType'][0] + ' - ' + client_data['carriername'][0]
    data_to_add.append({'IP':client_data['IpAddress'][0], 'CarrierTypeAndName':carrier})
to_add = pd.DataFrame.from_dict(data_to_add)

In [ ]:
df = pd.concat([transactionaldata, to_add], axis=1)[['CashoutAmount','tip','TmStamp','DeviceModel','CarrierTypeAndName','IP']]

In [ ]:
df['CashoutAmount'] = pd.Series(["${0:.2f}".format(val) for val in df['CashoutAmount']], index = df.index)
df.columns = ['Amount','Tip' , 'Logged in to cash out', 'Device', 'Carrier Type/Name', 'IP Address']

In [ ]:
df.to_clipboard(index = False, header = False)

In [ ]:
ahsql.disconnect()

In [ ]:
clientsql.disconnect()